In [15]:
import tensorflow as tf

In [4]:
from tensorflow.keras.models import Sequential

In [5]:
from tensorflow.keras.layers import LSTM,Dense,Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
import numpy as np
import regex as re

In [7]:
def file_load(file_path):
  with open(file_path,'r') as file:
    text=file.read()
  sentences=[sentence.strip() for sentence in re.split(r'(?<=[.!?])\s+',text) if sentence.strip()]
  return sentences

In [9]:
file_path=r'C:\Users\Personal\Documents\pizza1.txt'
text_data=file_load(file_path)

In [10]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(text_data)
len(tokenizer.word_index)

686

In [11]:
total_words=len(tokenizer.word_index)+1
total_words

687

In [12]:
input_sequences=[]
for line in text_data:
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram_sequences=token_list[:i+1]
    input_sequences.append(n_gram_sequences)

In [13]:
max_seq_len=max([len(seq) for seq in input_sequences])
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_seq_len,padding='pre'))
x,y=input_sequences[:,:-1],input_sequences[:,-1]

In [16]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [17]:
model=Sequential()
model.add(Embedding(total_words,10,input_length=max_seq_len-1))
model.add(LSTM(128))
model.add(Dense(total_words,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
model.fit(x,y,epochs=200,verbose=1)

Epoch 1/200
51/51 [==============================] - 1s 27ms/step - loss: 0.5888 - accuracy: 0.9232
Epoch 2/200
51/51 [==============================] - 1s 26ms/step - loss: 0.5771 - accuracy: 0.9201
Epoch 3/200
51/51 [==============================] - 1s 26ms/step - loss: 0.5617 - accuracy: 0.9226
Epoch 4/200
51/51 [==============================] - 1s 26ms/step - loss: 0.5466 - accuracy: 0.9257
Epoch 5/200
51/51 [==============================] - 1s 26ms/step - loss: 0.5360 - accuracy: 0.9244
Epoch 6/200
51/51 [==============================] - 1s 26ms/step - loss: 0.5218 - accuracy: 0.9312
Epoch 7/200
51/51 [==============================] - 1s 25ms/step - loss: 0.5091 - accuracy: 0.9312
Epoch 8/200
51/51 [==============================] - 1s 25ms/step - loss: 0.4950 - accuracy: 0.9318
Epoch 9/200
51/51 [==============================] - 1s 25ms/step - loss: 0.4844 - accuracy: 0.9318
Epoch 10/200
51/51 [==============================] - 1s 26ms/step - loss: 0.4723 - accuracy: 0.9312

In [32]:
seed_text='In addition to its culinary impact, pizza has'
next_words=7
for _ in range(next_words):
  token_list=tokenizer.texts_to_sequences([seed_text])[0]
  token_list=pad_sequences([token_list],maxlen=max_seq_len-1,padding='pre')
  predicted_probs=model.predict(token_list)
  predicted_word=tokenizer.index_word[np.argmax(predicted_probs)]
  seed_text+=' '+predicted_word
print(f"Text with next {next_words} words: ",seed_text)

1/1 [==============================] - 0s 29ms/step
Text with next 7 words:  In addition to its culinary impact, pizza has left an indelible mark on popular culture
